In [277]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import print_function, division
from scipy import stats
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from Utility.IO import *
import math

ModuleNotFoundError: No module named 'Utility'

In [1]:
def clean_and_prepare():
    # Load csv file into dataframe
    sc96 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1996_97_PP.csv")
    sc97 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1997_98_PP.csv")
    sc98 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1998_99_PP.csv")
    sc99 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1999_00_PP.csv")
    sc00 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2000_01_PP.csv")
    sc01 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2001_02_PP.csv")
    sc02 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2002_03_PP.csv")
    sc03 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2003_04_PP.csv")
    sc04 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2004_05_PP.csv")
    sc05 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2005_06_PP.csv")
    sc06 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2006_07_PP.csv")
    sc07 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2007_08_PP.csv")
    sc08 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2008_09_PP.csv")
    sc09 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2009_10_PP.csv")
    sc10 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2010_11_PP.csv")
    sc11 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2011_12_PP.csv")
    sc12 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2012_13_PP.csv")
    sc13 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2013_14_PP.csv")
    sc14 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2014_15_PP.csv")
    
    # Load Dictionary
    dic_file=pd.ExcelFile("../data/CollegeScorecardDataDictionary.xlsx")
    dic_init = dic_file.parse("data_dictionary")
    dic_init.head()
    dic_init.columns
    
    # Clean Dictionary
    dic=dic_init.ix[dic_init['VARIABLE NAME'].notnull()]
    
    sc_init=[sc96,sc97,sc98,sc99,sc00,sc01,sc02,sc03,sc04,sc05,sc06,sc07,sc08,sc09,sc10,sc11,sc12,sc13,sc14]
    
    # Add year variable
    for i in range(len(sc_init)):
        sc_init[i]['Year']=i+1996
    
    # Merge dataset with valid repayment information (2007-2014)
    sc_merge=pd.concat(sc_init[11:19])
    
    # Pick x and y variables
    scx_merge=sc_merge.loc[:,dic['VARIABLE NAME'][dic["dev-category"]!="repayment"]]
    scy_merge=sc_merge.loc[:,['RPY_1YR_RT','RPY_3YR_RT','RPY_5YR_RT', 'RPY_7YR_RT'] ]
    
    # Combine x and y variables
    sc=pd.concat([scy_merge,scx_merge,sc_merge['Year']],axis=1)
    
    # Drop empty row and column
    sc_drop=sc.dropna(how="all",axis=1)
    sc_drop=sc_drop.dropna(how="all",axis=0)
    
    # Save it to local file
    sc_drop.to_csv('../data/sc_final.csv')
    
    return sc_drop

In [5]:
# Example
sc=clean_and_prepare()
# after creating .csv file, we can obtain data from file directly
sc= pd.read_csv("../data/sc_final.csv",encoding="ISO-8859-1")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,6,442,451,598,607,611,620,624,633,789,802,806,815,893,906,971,984,997,1153,1166,1407,1408,1411,1425,1431,1432,1433,1437,1438,1439,1440,1445,1446,1447,1451,1452,1453,1454,1459,1460,1461,1465,1466,1467,1468,1473,1474,1475,1479,1480,1481,1482,1487,1488,1489,1501,1502,1537,1538,1539,1540,1541,1542,1603,1606,1609,1610,1611,1613,1614,1615,1616,1688,1689,1690,1691,1692,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.r

In [6]:
sc.RPY_1YR_RT=pd.Series(sc.RPY_1YR_RT).convert_objects(convert_numeric=True)
sc.RPY_3YR_RT=pd.Series(sc.RPY_3YR_RT).convert_objects(convert_numeric=True)
sc.RPY_5YR_RT=pd.Series(sc.RPY_5YR_RT).convert_objects(convert_numeric=True)
sc.RPY_7YR_RT=pd.Series(sc.RPY_7YR_RT).convert_objects(convert_numeric=True)
sc.Year=sc.Year.astype('object')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


In [8]:
# Load Dictionary
dic_file=pd.ExcelFile("../data/CollegeScorecardDataDictionary.xlsx")
dic_init = dic_file.parse("data_dictionary")
    
# Clean Dictionary
dic=dic_init.ix[dic_init['VARIABLE NAME'].notnull()]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [37]:
dic.head()

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VARIABLE NAME,VALUE,LABEL,SOURCE,NOTES
0,Unit ID for institution,root,id,integer,UNITID,NaN,NaN,IPEDS,Shown/used on consumer website.
1,8-digit OPE ID for institution,root,ope8_id,integer,OPEID,NaN,NaN,IPEDS,Shown/used on consumer website.
2,6-digit OPE ID for institution,root,ope6_id,integer,OPEID6,NaN,NaN,IPEDS,Shown/used on consumer website.
3,Institution name,school,name,autocomplete,INSTNM,NaN,NaN,IPEDS,Shown/used on consumer website.
4,City,school,city,autocomplete,CITY,NaN,NaN,IPEDS,Shown/used on consumer website.


In [39]:
sc.RPY_1YR_RT=pd.Series(sc.RPY_1YR_RT).convert_objects(convert_numeric=True)
sc.RPY_3YR_RT=pd.Series(sc.RPY_3YR_RT).convert_objects(convert_numeric=True)
sc.RPY_5YR_RT=pd.Series(sc.RPY_5YR_RT).convert_objects(convert_numeric=True)
sc.RPY_7YR_RT=pd.Series(sc.RPY_7YR_RT).convert_objects(convert_numeric=True)
sc.FEMALE_ENRL_ORIG_YR2_RT=pd.Series(sc.FEMALE_ENRL_ORIG_YR2_RT).convert_objects(convert_numeric=True)
sc.DEP_INC_PCT_LO=pd.Series(sc.DEP_INC_PCT_LO).convert_objects(convert_numeric=True)
sc.DEP_INC_PCT_H1=pd.Series(sc.DEP_INC_PCT_H1).convert_objects(convert_numeric=True)
sc.DEP_INC_PCT_H2=pd.Series(sc.DEP_INC_PCT_H2).convert_objects(convert_numeric=True)
sc.PAR_ED_PCT_PS=pd.Series(sc.PAR_ED_PCT_PS).convert_objects(convert_numeric=True)
sc.DEP_INC_AVG=pd.Series(sc.DEP_INC_AVG).convert_objects(convert_numeric=True)
sc.PELL_EVER=pd.Series(sc.PELL_EVER).convert_objects(convert_numeric=True)
sc.PAR_ED_PCT_1STGEN=pd.Series(sc.PAR_ED_PCT_1STGEN).convert_objects(convert_numeric=True)
sc.C200_4_POOLED_SUPP=pd.Series(sc.C200_4_POOLED_SUPP).convert_objects(convert_numeric=True)
sc.Year=sc.Year.astype('object')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.
C:\Progra

In [43]:
def get_rpyrt_by_class(y=1):
    index={'1':1,'3':2,'5':3,'7':4}
    sc= pd.read_csv("../data/sc_final.csv",encoding="ISO-8859-1")
    yname='RPY_'+ str(y) +'YR_RT'
    sc_sub=pd.concat([sc.iloc[:,[index[str(y)]]],sc.drop(sc.columns[range(0,5)], axis=1)],axis=1)
    sc_sub=sc_sub.convert_objects(convert_numeric=True)
    sc_sub=sc_sub.dropna(subset=[yname])
    pro=0.3
    sc_sub=sc_sub.dropna(axis=0,how='all')
    sc_sub=sc_sub.dropna(axis=1,thresh=np.ceil(sc_sub.shape[1]*(1-pro)))
    return sc_sub

In [44]:
sc_rpt1=get_rpyrt_by_class(1)
sc_rpt3=get_rpyrt_by_class(3)
sc_rpt5=get_rpyrt_by_class(5)
sc_rpt7=get_rpyrt_by_class(7)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,2,3,4,6,11,12,13,14,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,6

In [211]:
def get_corplot_by_class(y=1,thre=200):
    yname='RPY_'+ str(y) +'YR_RT'
    yname=math.log(1/(1-yname))
    sc_rpt=get_rpyrt_by_class(y)
    cor=sc_rpt.corr()
    corr=abs(cor).sort_values([yname],ascending=0)[yname]
    sc_cor=cor.loc[corr.index[0:thre],corr.index[0:thre]]
    fig, ax = plt.subplots(figsize=(10,7)) 
    sns.heatmap(abs(sc_cor),ax=ax, xticklabels=sc_cor.columns,yticklabels=sc_cor.columns)
    return cor.loc[corr.index[0:thre],corr.index[0:thre]]

In [214]:
sc_rpt=get_rpyrt_by_class(1)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,2,3,4,6,11,12,13,14,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,6

In [221]:
sc_rpt=sc_rpt.drop(sc_rpt.columns[range(1,7)], axis=1)
cor=sc_rpt.corr()

In [225]:
corr=abs(cor).sort_values(['RPY_1YR_RT'],ascending=0)['RPY_1YR_RT']

In [265]:
x = cor.loc[corr.index[0:200],corr.index[0:10]]
x.head()

,RPY_1YR_RT,INC_PCT_H1,PELL_EVER,INC_PCT_LO,FAMINC,DEP_INC_PCT_H1,MD_FAMINC,DEP_INC_AVG,DEP_INC_PCT_LO,DEP_INC_PCT_H2
RPY_1YR_RT,1.000000,0.815670,-0.808683,-0.801793,0.787495,0.784575,0.773515,0.773407,-0.763898,0.751378
INC_PCT_H1,0.815670,1.000000,-0.870945,-0.958296,0.903737,0.920023,0.916551,0.866466,-0.911108,0.827679
PELL_EVER,-0.808683,-0.870945,1.000000,0.880185,-0.885932,-0.859208,-0.858237,-0.921452,0.879351,-0.880253
INC_PCT_LO,-0.801793,-0.958296,0.880185,1.000000,-0.952283,-0.875856,-0.947105,-0.902501,0.925530,-0.862374
FAMINC,0.787495,0.903737,-0.885932,-0.952283,1.000000,0.819237,0.980613,0.942381,-0.874513,0.927480


In [266]:
sc_sub=sc.drop(sc.columns[range(6,12)], axis=1)
sc_sub=sc_sub.convert_objects(convert_numeric=True)
sc_sub.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


,Unnamed: 0,RPY_1YR_RT,RPY_3YR_RT,RPY_5YR_RT,RPY_7YR_RT,UNITID,ACCREDAGENCY,INSTURL,NPCURL,SCH_DEG,...,C100_L4,D100_L4,TRANS_4,DTRANS_4,TRANS_L4,DTRANS_L4,ICLEVEL,UGDS_MEN,UGDS_WOMEN,Year
0,0,0.477044,NaN,NaN,NaN,100654,NaN,NaN,NaN,3.0,...,NaN,NaN,0.000000,1038.0,NaN,NaN,1.0,0.4709,0.5291,2007
1,1,0.608185,NaN,NaN,NaN,100663,NaN,NaN,NaN,3.0,...,NaN,NaN,0.234477,1224.0,NaN,NaN,1.0,0.3956,0.6044,2007
2,2,0.657534,NaN,NaN,NaN,100690,NaN,NaN,NaN,3.0,...,NaN,NaN,0.600000,5.0,NaN,NaN,1.0,0.4752,0.5248,2007
3,3,0.657205,NaN,NaN,NaN,100706,NaN,NaN,NaN,3.0,...,NaN,NaN,0.331081,592.0,NaN,NaN,1.0,0.5169,0.4831,2007
4,4,0.432048,NaN,NaN,NaN,100724,NaN,NaN,NaN,3.0,...,NaN,NaN,0.000000,1263.0,NaN,NaN,1.0,0.4026,0.5974,2007


In [267]:
sd=pd.DataFrame(np.std(sc_sub[x.index]))
new = sd[sd[0] >0.15]  
new.head()

,0
RPY_1YR_RT,0.203795
PELL_EVER,0.173060
INC_PCT_LO,0.195161
FAMINC,22931.272806
MD_FAMINC,19618.771583


In [268]:
data=sc_sub[new.index]
y_var=data.RPY_1YR_RT
x_var=data.drop('RPY_1YR_RT', 1)
x_var=x_var.fillna(x_var.mean())
x_var=(x_var-x_var.mean())/x_var.std()
y_var=y_var.fillna(y_var.mean())
y_var=np.log(y_var/(1-y_var))
y_var

0       -0.091888
1        0.439691
2        0.652325
3        0.650864
4       -0.273501
5        0.697143
6       -0.068993
7        0.850151
8        0.359152
9        1.501019
10       1.634131
11      -0.561469
12      -0.173665
13      -0.077663
14       0.245122
15       0.024293
16       0.278295
17      -0.173665
18      -0.173665
19       0.287682
20      -0.173665
21      -0.596123
22       0.955511
23       0.798508
24      -0.173665
25       0.469462
26      -0.173665
27       0.019556
28      -0.008625
29       0.747214
           ...   
59373   -0.173665
59374   -0.173665
59375   -0.173665
59376   -0.173665
59377   -0.173665
59378   -0.173665
59379   -0.173665
59380   -0.173665
59381   -0.173665
59382   -0.173665
59383   -0.173665
59384   -0.173665
59385   -0.173665
59386   -0.173665
59387   -0.173665
59388   -0.173665
59389   -0.173665
59390   -0.173665
59391   -0.173665
59392   -0.173665
59393   -0.173665
59394   -0.173665
59395   -0.173665
59396   -0.173665
59397   -0

In [272]:
model = LinearRegression()

In [273]:
from sklearn.cross_validation import cross_val_score
cv = cross_val_score(model, x_var, y_var, cv=10)
cv.mean()

-1.3444862346530193e+31

In [271]:
cross_val_score(model, x_var, y_var, cv=10)

array([  4.79667597e-01,   6.28141595e-01,   6.54111279e-01,
         6.41816913e-01,   6.31731707e-01,   6.71432861e-01,
         6.51795409e-01,   1.86742642e-01,  -8.83384542e+31,
        -4.61101693e+31])

In [257]:
np.exp(cv.mean())

0.0

In [276]:
max(x_var)

'UGDS_WHITENH'

In [251]:
max(y_var)

3.0376964723536095

In [252]:
min(y_var)

-3.3833303804799399

In [199]:
cross_val_score(model, x_var, y_var, cv=10)

array([  4.40039276e-01,   6.21216139e-01,   6.53809735e-01,
         6.48182725e-01,   6.47025554e-01,   6.91842016e-01,
         6.75441879e-01,   1.87413521e-01,  -9.40878571e+29,
        -5.13904792e+29])

In [171]:
lm1 = smf.ols(formula='RPY_1YR_RT ~ PELL_EVER + INC_PCT_LO +  FAMINC + MD_FAMINC + DEP_INC_AVG + DEP_INC_PCT_LO + FEMALE_ENRL_ORIG_YR2_RT + MALE_ENRL_ORIG_YR2_RT + PELL_ENRL_ORIG_YR2_RT + NOT1STGEN_ENRL_ORIG_YR2_RT + NOPELL_ENRL_ORIG_YR2_RT + NOLOAN_ENRL_ORIG_YR2_RT + ENRL_ORIG_YR2_RT', data=sc_sub).fit()
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             RPY_1YR_RT   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     2778.
Date:                Wed, 29 Nov 2017   Prob (F-statistic):               0.00
Time:                        12:45:39   Log-Likelihood:                 10585.
No. Observations:               11031   AIC:                        -2.114e+04
Df Residuals:                   11017   BIC:                        -2.104e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      1.2267      0.027     45.153      0.000       1.173       1.280
PELL_EVER                     -0.9096      0.027    -33.567      0.000      -0.963      -0.856
INC_PCT_LO                     0.4559      0.053      8.605      0.000       0.352       0.560
FAMINC                      6.272e-06   6.26e-07     10.019      0.000    5.04e-06     7.5e-06
MD_FAMINC                  -6.552e-07   3.04e-07     -2.158      0.031   -1.25e-06   -6.01e-08
DEP_INC_AVG                -7.603e-06   4.93e-07    -15.413      0.000   -8.57e-06   -6.64e-06
DEP_INC_PCT_LO                -0.7032      0.046    -15.370      0.000      -0.793      -0.613
FEMALE_ENRL_ORIG_YR2_RT       -0.7076      0.142     -4.974      0.000      -0.986      -0.429
MALE_ENRL_ORIG_YR2_RT         -1.0458      0.095    -11.046      0.000      -1.231      -0.860
PELL_ENRL_ORIG_YR2_RT         -0.5141      0.048    -10.749      0.000      -0.608      -0.420
NOT1STGEN_ENRL_ORIG_YR2_RT    -0.5585      0.070     -8.001      0.000      -0.695      -0.422
NOPELL_ENRL_ORIG_YR2_RT       -0.4268      0.028    -15.293      0.000      -0.481      -0.372
NOLOAN_ENRL_ORIG_YR2_RT        0.3330      0.012     26.738      0.000       0.309       0.357
ENRL_ORIG_YR2_RT               3.3592      0.250     13.413      0.000       2.868       3.850
==============================================================================
Omnibus:                      276.324   Durbin-Watson:                   1.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              355.854
Skew:                           0.310   Prob(JB):                     5.34e-78
Kurtosis:                       3.624   Cond. No.                     3.02e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.02e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""